In [ ]:
# please run the below line if you don't have the packages
# !pip install jupysql duckdb duckdb-engine --quiet
# !pip install pandas --quiet

In [1]:
import pandas as pd

In [2]:
%load_ext sql
%sql duckdb://

In [3]:
# loading all the tables
album = pd.read_csv('source/album.csv')
artist = pd.read_csv('source/artist.csv')
customer = pd.read_csv('source/customer.csv')
employee = pd.read_csv('source/employee.csv')
genre = pd.read_csv('source/genre.csv')
invoice_line = pd.read_csv('source/invoice_line.csv')
invoice = pd.read_csv('source/invoice.csv')
media_type = pd.read_csv('source/media_type.csv')
playlist_track = pd.read_csv('source/playlist_track.csv')
playlist = pd.read_csv('source/playlist.csv')
track = pd.read_csv('source/track.csv')

### 1.  Who is the senior most employee based on job title? 

In [4]:
%%sql
SELECT levels , GROUP_CONCAT(title)
FROM employee
GROUP BY levels

*  duckdb://
Done.


levels,group_concat(title)
L6,General Manager
L4,Sales Manager
L1,"Sales Support Agent,Sales Support Agent,Sales Support Agent"
L3,IT Manager
L2,"IT Staff,IT Staff"
L7,Senior General Manager


In [5]:
%%sql
SELECT *
FROM employee 
WHERE levels = 'L7'

*  duckdb://
Done.


employee_id,last_name,first_name,title,reports_to,levels,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
9,Madan,Mohan,Senior General Manager,None,L7,26-01-1961 00:00,14-01-2016 00:00,1008 Vrinda Ave MT,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,madan.mohan@chinookcorp.com


### 2.  Which countries have the most Invoices? 

In [6]:
%%sql
SELECT billing_country,COUNT(*) as invoices_count
FROM invoice
GROUP BY billing_country
ORDER BY invoices_count DESC
LIMIT 10 --say we look into top 10

*  duckdb://
Done.


billing_country,invoices_count
USA,131
Canada,76
Brazil,61
France,50
Germany,41
Czech Republic,30
Portugal,29
United Kingdom,28
India,21
Ireland,13


### 3.  What are top 3 values of total invoice? 

In [7]:
%%sql
SELECT total
FROM invoice 
ORDER BY total DESC
LIMIT 3

*  duckdb://
Done.


total
23.76
19.8
19.8


### 4.  Which  city  has  the  best  customers?  We  would  like  to  throw  a  promotional  Music Festival in the city we made the most money. Write a query that returns one city that has the highest sum of invoice totals. Return both the city name & sum of all invoice totals 

In [8]:
%%sql
SELECT city, ROUND(sum(i.total),2) AS sum_all
FROM customer c LEFT JOIN invoice i
ON c.customer_id = i.customer_id
GROUP BY city
ORDER BY sum_all DESC
LIMIT 1

*  duckdb://
Done.


city,sum_all
Prague,273.24


### 5.  Who  is  the  best  customer?  The  customer  who  has  spent  the  most  money  will  be declared the best customer. Write a query that returns the person who has spent the most money

In [9]:
%%sql
SELECT c.customer_id, i.invoice_id, c.first_name, c.last_name, i.total
FROM customer c LEFT JOIN invoice i
ON c.customer_id = i.customer_id
ORDER BY i.total DESC
LIMIT 1

*  duckdb://
Done.


customer_id,invoice_id,first_name,last_name,total
42,183,Wyatt,Girard,23.76


### 6.  Write  query  to  return  the  email,  first  name,  last  name,  &  Genre  of  all  Rock  Music listeners. Return your list ordered alphabetically by email starting with A

In [60]:
%%sql
SELECT DISTINCT c.email, c.first_name, c.last_name, g.name
FROM customer c
LEFT JOIN invoice i ON c.customer_id = i.customer_id
LEFT JOIN invoice_line il ON i.invoice_id = il.invoice_id 
LEFT JOIN track t ON il.track_id = t.track_id 
LEFT JOIN genre g ON t.genre_id = g.genre_id
WHERE g.name = 'Rock'
ORDER BY c.email
LIMIT 10 -- let's limit it to top 10

*  duckdb://
Done.


email,first_name,last_name,name
aaronmitchell@yahoo.ca,Aaron,Mitchell,Rock
alero@uol.com.br,Alexandre,Rocha,Rock
astrid.gruber@apple.at,Astrid,Gruber,Rock
bjorn.hansen@yahoo.no,Bjørn,Hansen,Rock
camille.bernard@yahoo.fr,Camille,Bernard,Rock
daan_peeters@apple.be,Daan,Peeters,Rock
diego.gutierrez@yahoo.ar,Diego,Gutiérrez,Rock
dmiller@comcast.com,Dan,Miller,Rock
dominiquelefebvre@gmail.com,Dominique,Lefebvre,Rock
edfrancis@yachoo.ca,Edward,Francis,Rock


### 7.  Let's  invite  the  artists  who  have  written  the  most  rock  music  in  our  dataset.  Write  a query that returns the Artist name and total track count of the top 10 rock bands

In [49]:
%%sql
SELECT artist.name, count(*) as total_tracks
FROM artist 
JOIN album ON artist.artist_id = album.artist_id
JOIN track ON album.album_id = track.album_id
JOIN genre ON track.genre_id = genre.genre_id
WHERE genre.name = 'Rock'
GROUP BY artist.name
ORDER BY total_tracks DESC
LIMIT 10

*  duckdb://
Done.


name,total_tracks
Led Zeppelin,114
U2,112
Deep Purple,92
Iron Maiden,81
Pearl Jam,54
Van Halen,52
Queen,45
The Rolling Stones,41
Creedence Clearwater Revival,40
Kiss,35


### 8.  Return all the track names that have a song length longer than the average song length. Return  the Name  and  Milliseconds  for  each  track.  Order by  the  song  length with  the longest songs listed first

In [59]:
%%sql
SELECT name, milliseconds FROM track
WHERE milliseconds > (select AVG(milliseconds) FROM track)
ORDER BY milliseconds DESC
LIMIT 10 --let's limit it to top 10

*  duckdb://
Done.


name,milliseconds
Occupation / Precipice,5286953
Through a Looking Glass,5088838
"Greetings from Earth, Pt. 1",2960293
The Man With Nine Lives,2956998
"Battlestar Galactica, Pt. 2",2956081
"Battlestar Galactica, Pt. 1",2952702
Murder On the Rising Star,2935894
"Battlestar Galactica, Pt. 3",2927802
Take the Celestra,2927677
Fire In Space,2926593


### 9.  We want to find out the most popular and least popular music Genre for each country. We determine the most popular genre as the genre with the highest amount of purchases and lowest amount of purchases for least popular genre.

In [112]:
%%sql
WITH cte as(
SELECT invoice.billing_country, genre.name, SUM(invoice.total) as sum_total
FROM invoice
JOIN invoice_line ON invoice.invoice_id = invoice_line.invoice_id
JOIN track ON invoice_line.track_id = track.track_id
JOIN genre ON track.genre_id = genre.genre_id
GROUP BY invoice.billing_country, genre.name
ORDER BY invoice.billing_country,sum_total)

SELECT billing_country,name,ROUND(sum_total,2) as "total_purchase_sum",
CASE
WHEN ROUND(sum_total,2) = (SELECT ROUND(MAX(sum_total),2) FROM cte a WHERE a.billing_country = alias.billing_country)  THEN 'Most Popular' 
WHEN ROUND(sum_total,2) = (SELECT ROUND( MIN(sum_total),2) FROM cte b WHERE b.billing_country = alias.billing_country)  THEN 'Least Popular' 
END popular_label
FROM(
SELECT * ,(sum_total= FIRST_VALUE(sum_total) OVER(PARTITION BY billing_country ORDER BY sum_total DESC)) as boolean_popular,
(sum_total= FIRST_VALUE(sum_total) OVER(PARTITION BY billing_country ORDER BY sum_total)) as boolean_least
FROM cte 
ORDER BY billing_country) alias
WHERE boolean_popular = 1 or boolean_least = 1
ORDER BY billing_country,total_purchase_sum

*  duckdb://
Done.


billing_country,name,total_purchase_sum,popular_label
Argentina,Easy Listening,0.99,Least Popular
Argentina,Alternative & Punk,203.94,Most Popular
Australia,Reggae,1.98,Least Popular
Australia,Alternative & Punk,360.36,Most Popular
Austria,R&B/Soul,2.97,Least Popular
Austria,Rock,341.55,Most Popular
Belgium,R&B/Soul,5.94,Least Popular
Belgium,Alternative,5.94,Least Popular
Belgium,Electronica/Dance,5.94,Least Popular
Belgium,Rock,229.68,Most Popular


### 10. Write a query that determines the customer that has spent the most on music for each country. Write a query that returns the country along with the top customer and how much  they  spent.  For  countries  where  the  top  amount  spent  is  shared,  provide  all customers who spent this amount 

In [172]:
%%sql
WITH cte AS(
SELECT i.customer_id, i.billing_country, sum(total) AS sum_total,
RANK() OVER(PARTITION BY i.billing_country ORDER BY sum(total) DESC) as rnk
FROM customer c
JOIN invoice i ON c.customer_id = i.customer_id
GROUP BY i.customer_id, i.billing_country
ORDER BY i.billing_country,sum_total DESC)

SELECT customer_id,billing_country,ROUND(sum_total,2) sum_total FROM cte
WHERE rnk = 1

*  duckdb://
Done.


customer_id,billing_country,sum_total
56,Argentina,39.6
55,Australia,81.18
7,Austria,69.3
8,Belgium,60.39
1,Brazil,108.9
3,Canada,99.99
57,Chile,97.02
5,Czech Republic,144.54
9,Denmark,37.62
44,Finland,79.2
